In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Huntingtons_Disease"
cohort = "GSE135589"

# Input paths
in_trait_dir = "../../input/GEO/Huntingtons_Disease"
in_cohort_dir = "../../input/GEO/Huntingtons_Disease/GSE135589"

# Output paths
out_data_file = "../../output/preprocess/Huntingtons_Disease/GSE135589.csv"
out_gene_data_file = "../../output/preprocess/Huntingtons_Disease/gene_data/GSE135589.csv"
out_clinical_data_file = "../../output/preprocess/Huntingtons_Disease/clinical_data/GSE135589.csv"
json_path = "../../output/preprocess/Huntingtons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"2-year Longitudinal Analyses of Peripheral Blood RNA in Huntington's Disease gene-positive and control subjects"
!Series_summary	"Samples collected in the course of the TRACK HD study and used to analyse association between gene expression in blood and symptomatic progression of Huntington's disease"
!Series_summary	"Longitudinal analysis of UHDRS TMS versus gene expression in individual subjects followed by cross-sectional comparisons across phenotype groups at each timepoint"
!Series_overall_design	"Year 1: 24 Controls, 17 Huntington's mutation-positive presymptomatic far from age of predicted onset (Pre-A), 18 Huntington's mutation-positive presymptomatic near to age of predicted onset (Pre-B), 18 Manifest Huntington's disease Stage 1 (zHD1), 19 Manifest Huntington's disease Stage 2 (zHD2); Year 3: 20 Controls, 15 pre-A, 15 pre-B, 16 zHD1, 16 zHD 2"
!Series_overall_design	"contributor: TRACK-HD Investigators"
Sample Characteristics Dictionary:


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset includes gene expression data for Huntington's disease
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Huntington's Disease), key 4 contains "disease stage" information
trait_row = 4
# For age, key 2 contains "age at year 1" information
age_row = 2
# For gender, key 1 contains "Sex" information
gender_row = 1

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert disease stage information to binary (0: Control, 1: HD affected)"""
    if value is None:
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Binary classification: Control (0) vs. any HD stage (1)
    if 'Control' in value:
        return 0
    elif 'preHD' in value or 'zHD' in value:
        return 1
    else:
        return None

def convert_age(value: str) -> float:
    """Convert age information to continuous value"""
    if value is None:
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> int:
    """Convert gender information to binary (0: Female, 1: Male)"""
    if value is None:
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'Female' in value:
        return 0
    elif 'Male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    print("Trait data is available, but we need the actual clinical data from previous steps to extract features.")
    print(f"Trait row: {trait_row}, Age row: {age_row}, Gender row: {gender_row}")
    print("For trait conversion, Control is mapped to 0, preHD/zHD stages are mapped to 1.")
    print("For age conversion, the numeric age value is extracted.")
    print("For gender conversion, Female is mapped to 0, Male is mapped to 1.")
    
    # We've identified the relevant rows and conversion functions,
    # but need the actual clinical_data matrix to proceed with extraction
    print("Note: The geo_select_clinical_features function requires the actual clinical data matrix.")
    print("The sample characteristics dictionary only shows unique values, not the matrix needed for processing.")
    
    # In a real execution, we'd continue with actual clinical_data when available:
    # selected_clinical_df = geo_select_clinical_features(
    #     clinical_df=clinical_data,
    #     trait=trait,
    #     trait_row=trait_row,
    #     convert_trait=convert_trait,
    #     age_row=age_row,
    #     convert_age=convert_age,
    #     gender_row=gender_row,
    #     convert_gender=convert_gender
    # )
    #
    # selected_clinical_df.to_csv(out_clinical_data_file, index=False)
else:
    print("No trait data available, skipping clinical feature extraction.")


Trait data is available, but we need the actual clinical data from previous steps to extract features.
Trait row: 4, Age row: 2, Gender row: 1
For trait conversion, Control is mapped to 0, preHD/zHD stages are mapped to 1.
For age conversion, the numeric age value is extracted.
For gender conversion, Female is mapped to 0, Male is mapped to 1.
Note: The geo_select_clinical_features function requires the actual clinical data matrix.
The sample characteristics dictionary only shows unique values, not the matrix needed for processing.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Huntingtons_Disease/GSE135589/GSE135589_series_matrix.txt.gz


Gene data shape: (54675, 178)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the identifiers shown (like '1007_s_at', '1053_at', etc.), these appear to be 
# Affymetrix probe IDs rather than standard human gene symbols.
# Affymetrix probe IDs typically have this format with numbers followed by "_at" or similar suffixes.
# These IDs will need to be mapped to human gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene identifiers and gene symbols
id_column = 'ID'  # Contains probe IDs matching the gene expression data index
gene_symbol_column = 'Gene Symbol'  # Contains the corresponding gene symbols

print(f"Using probe ID column: '{id_column}' and gene symbol column: '{gene_symbol_column}'")

# 2. Get a gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, id_column, gene_symbol_column)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Sample of gene mapping data:")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("Sample of gene expression data (first 5 genes, first 3 samples):")
print(gene_data.iloc[:5, :3])

# 4. Save the gene expression data to a CSV file
# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Using probe ID column: 'ID' and gene symbol column: 'Gene Symbol'


Gene mapping dataframe shape: (45782, 2)
Sample of gene mapping data:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A


Gene expression data shape after mapping: (21278, 178)
Sample of gene expression data (first 5 genes, first 3 samples):
          GSM4020181  GSM4020182  GSM4020183
Gene                                        
A1BG        6.001051    5.641555    5.983165
A1BG-AS1    8.457736    8.255808    8.650942
A1CF        7.334571    8.234268    8.322526
A2M        13.597006   14.135605   13.992524
A2M-AS1     9.956841   10.146245    9.651576


Gene expression data saved to: ../../output/preprocess/Huntingtons_Disease/gene_data/GSE135589.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. First, extract and save the clinical data since it's missing
# Get the SOFT and matrix file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get the background info and clinical data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define the conversion functions from Step 2
def convert_trait(value: str) -> int:
    """Convert disease stage information to binary (0: Control, 1: HD affected)"""
    if value is None:
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Binary classification: Control (0) vs. any HD stage (1)
    if 'Control' in value:
        return 0
    elif 'preHD' in value or 'zHD' in value:
        return 1
    else:
        return None

def convert_age(value: str) -> float:
    """Convert age information to continuous value"""
    if value is None:
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> int:
    """Convert gender information to binary (0: Female, 1: Male)"""
    if value is None:
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'Female' in value:
        return 0
    elif 'Male' in value:
        return 1
    else:
        return None

# Extract clinical features with the identified rows from Step 2
trait_row = 4
age_row = 2
gender_row = 1

# Process and save clinical data
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to: {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 2. Normalize gene symbols using synonym information from NCBI
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene identifiers:")
print(gene_data.index[:10])

# Save the normalized gene data
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
# First check how many samples have missing trait values
if trait in linked_data.columns:
    missing_trait = linked_data[trait].isna().sum()
    print(f"Samples with missing trait values: {missing_trait} out of {len(linked_data)}")

# Check gene missing value percentages
gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
gene_missing_pct = linked_data[gene_cols].isna().mean()
genes_to_keep = gene_missing_pct[gene_missing_pct <= 0.2].index
print(f"Genes with ≤20% missing values: {len(genes_to_keep)} out of {len(gene_cols)}")

# Check sample missing value percentages
if len(gene_cols) > 0:
    sample_missing_pct = linked_data[gene_cols].isna().mean(axis=1)
    samples_to_keep = sample_missing_pct[sample_missing_pct <= 0.05].index
    print(f"Samples with ≤5% missing gene values: {len(samples_to_keep)} out of {len(linked_data)}")

# Apply missing value handling
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
trait_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This dataset contains gene expression data from blood samples of Huntington's Disease patients and controls, with clinical annotations including disease stage, age, gender, and CAG repeat count."
is_gene_available = len(gene_data) > 0
is_trait_available = trait in linked_data.columns
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=trait_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable and linked_data_clean.shape[0] > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Clinical data saved to: ../../output/preprocess/Huntingtons_Disease/clinical_data/GSE135589.csv
Clinical data preview:
{'GSM4020181': [0.0, 36.0, 0.0], 'GSM4020182': [0.0, 36.0, 0.0], 'GSM4020183': [0.0, 36.0, 0.0], 'GSM4020184': [0.0, 36.0, 0.0], 'GSM4020185': [0.0, 39.0, 1.0], 'GSM4020186': [0.0, 39.0, 1.0], 'GSM4020187': [0.0, 32.0, 1.0], 'GSM4020188': [0.0, 32.0, 1.0], 'GSM4020189': [0.0, 36.0, 1.0], 'GSM4020190': [0.0, 36.0, 1.0], 'GSM4020191': [0.0, 57.0, 1.0], 'GSM4020192': [0.0, 57.0, 1.0], 'GSM4020193': [0.0, 44.0, 0.0], 'GSM4020194': [0.0, 44.0, 0.0], 'GSM4020195': [0.0, 47.0, 0.0], 'GSM4020196': [0.0, 37.0, 0.0], 'GSM4020197': [0.0, 37.0, 0.0], 'GSM4020198': [0.0, 56.0, 0.0], 'GSM4020199': [0.0, 56.0, 0.0], 'GSM4020200': [0.0, 31.0, 0.0], 'GSM4020201': [0.0, 31.0, 0.0], 'GSM4020202': [0.0, 59.0, 0.0], 'GSM4020203': [0.0, 59.0, 0.0], 'GSM4020204': [0.0, 48.0, 0.0], 'GSM4020205': [0.0, 48.0, 0.0], 'GSM4020206': [0.0, 52.0, 1.0], 'GSM4020207': [0.0, 59.0, 0.0], 'GSM4020208': [0

Normalized gene data saved to: ../../output/preprocess/Huntingtons_Disease/gene_data/GSE135589.csv

Linking clinical and genetic data...
Linked data shape: (178, 19848)
Linked data preview (first 5 rows, 5 columns):
            Huntingtons_Disease   Age  Gender      A1BG  A1BG-AS1
GSM4020181                  0.0  36.0     0.0  6.001051  8.457736
GSM4020182                  0.0  36.0     0.0  5.641555  8.255808
GSM4020183                  0.0  36.0     0.0  5.983165  8.650942
GSM4020184                  0.0  36.0     0.0  5.641889  8.356846
GSM4020185                  0.0  39.0     1.0  5.861286  8.243407

Handling missing values...
Samples with missing trait values: 0 out of 178
Genes with ≤20% missing values: 19845 out of 19845
Samples with ≤5% missing gene values: 178 out of 178


Linked data shape after handling missing values: (178, 19848)

Checking for bias in dataset features...
For the feature 'Huntingtons_Disease', the least common label is '0.0' with 44 occurrences. This represents 24.72% of the dataset.
The distribution of the feature 'Huntingtons_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 39.0
  50% (Median): 44.0
  75%: 51.0
Min: 25.0
Max: 64.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 83 occurrences. This represents 46.63% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Huntingtons_Disease/cohort_info.json


Linked data saved to ../../output/preprocess/Huntingtons_Disease/GSE135589.csv
